In [2]:
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from  services.influencers_service import  InfluencersService
from shared.mongo import MongoConnection
#visualisation 
import pyLDAvis
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora
import json 
import glob
import nltk
from nltk.corpus import stopwords

2024-05-28 17:52:01,886 - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-05-28 17:52:01,887 - INFO - NumExpr defaulting to 8 threads.


In [3]:
mongo_connection=MongoConnection()
influencers_service = InfluencersService(mongo_connection)
bios=[]
posts=influencers_service.get_posts()
influencers=influencers_service.get_influencers()
for influencer in influencers:
    info_string=""
    if influencer['Bio']!= 'NULL':
        info_string+=influencer['Bio']
    if influencer['Category']!= 'NULL':
        info_string+=influencer['Category']
    if info_string!= "":
        bios.append(influencer['Bio']+" "+influencer['Category'])

2024-05-28 17:52:09,173 - INFO - MongoDB connection established successfully
2024-05-28 17:52:29,142 - INFO - Fetched 20316 posts
2024-05-28 17:52:33,392 - INFO - Fetched 38112 influencers


In [ ]:
def merge_content_posts_bios(posts,bios) : 
    #concatenation of captions of all posts + bios of all influencers
    merged_content = ""
    for post in posts:
         #extract caption text
         caption = post.get('caption')
         if (caption):
            merged_content += caption + " "
    for bio in bios:
        merged_content+=bio
    return merged_content

In [ ]:
def gen_words(text):
    words=[]
    new = gensim.utils.simple_preprocess(text, deacc=True)    
    words.append(new)
    return words

In [ ]:
def gen_bow(words):
    id2word = corpora.Dictionary(words)
    corpus=[]
    for word in words: 
        new = id2word.doc2bow(word)
        corpus.append(new)
    return  corpus

In [ ]:
def make_bigrams(texts,bigram):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts,bigram,trigram):
    return [trigram[bigram[doc]] for doc in texts]

In [ ]:
# posts=posts[0:3] #for rapid testing we only use three posts
merge=merge_content_posts_bios(posts,bios)
# print(merge)
words=gen_words(merge)
# print(words)

##generate bigrams and tigrams##
bigram_phrases = gensim.models.Phrases(words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[words], threshold=100)
bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)
data_bigrams = make_bigrams(words,bigram)
data_bigrams_trigrams = make_trigrams(data_bigrams,bigram,trigram)


In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
texts = data_bigrams_trigrams #for easier typing
custom_stopwords = {
    'com','gmail','creator','love','new','creators','dm','www','hotmail','dm','info','null','email','collab','contact','lover','celebrities','life','live','influencer',
}  # Add any additional stop words to this set
stopwords = STOPWORDS.union(custom_stopwords)
filtered_texts = []
for text in texts:
    filtered_text = [word for word in text if word not in stopwords]
    filtered_texts.append(filtered_text)
texts=filtered_texts
# print(texts)

In [ ]:
# # #TF-IDF REMOVAL
# from gensim.models import TfidfModel
# # 
id2word = corpora.Dictionary(texts)
# 
corpus = [id2word.doc2bow(text) for text in texts]
# tfidf = TfidfModel(corpus, id2word=id2word,smartirs='ntc')
# 
# low_value =0.0005
# words  = []
# words_missing_in_tfidf = []
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

# print(corpus)

In [ ]:
lda_model=gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=3,
    random_state=50,
    chunksize=1000,
    update_every=1,
    iterations=10,
    passes=1,
    alpha=3,
    eta="auto",
)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30,lambda_step=0.01)
vis

In [ ]:
coherence_model =gensim.models.CoherenceModel(model=lda_model, texts=texts,dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(coherence_score)

In [ ]:
# def Sort(sub_li):
#     sub_li.sort(key=lambda  x:x[1])
#     sub_li.reverse()
#     return sub_li

In [ ]:
# # lda_model.save("../models/influencer_lda_model.model")
# new_model=gensim.models.ldamodel.LdaModel.load("../models/influencer_lda_model.model")


In [ ]:
# test_doc=corpus[-1]
# vector=new_model[test_doc]
# sorted_vector=Sort(vector)
# print(sorted_vector)